### Rue Salvesen Ratings
https://penaltyblog.readthedocs.io/en/latest/ratings/massey_ratings.html

https://www.pro-football-reference.com/years/2022/games.htm


In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import penaltyblog as pb

In [3]:
import nfl_data_py as nfl

In [4]:
try:
    import nfl_data_py as nfl
except:
    ! pip install nfl_data_py

In [5]:
semana = 8
temporada = 2022
normalizacion = 1
MAX = 10
df = nfl.import_schedules([temporada])
df = df[df['week']<semana]

In [6]:
df = df.rename(columns={'away_team':'AwayTeam',
                                   'home_team':'HomeTeam',
                       'away_score':'FTHG',
                       'home_score':'FTAG'})


In [7]:
df[['FTHG','FTAG']]

,FTHG,FTAG
6137,31.0,10.0
6138,27.0,26.0
6139,26.0,24.0
6140,10.0,19.0
6141,23.0,20.0
...,...,...
6240,20.0,38.0
6241,37.0,23.0
6242,44.0,23.0
6243,10.0,16.0


In [8]:
df[df['AwayTeam']=='BUF']

,game_id,season,game_type,week,gameday,weekday,gametime,AwayTeam,FTHG,HomeTeam,...,wind,away_qb_id,home_qb_id,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium
6137,2022_01_BUF_LA,2022,REG,1,2022-09-08,Thursday,20:20,BUF,31.0,LA,...,NaN,00-0034857,00-0026498,Josh Allen,Matthew Stafford,Sean McDermott,Sean McVay,Carl Cheffers,LAX01,SoFi Stadium
6173,2022_03_BUF_MIA,2022,REG,3,2022-09-25,Sunday,13:00,BUF,19.0,MIA,...,NaN,00-0034857,00-0036212,Josh Allen,Tua Tagovailoa,Sean McDermott,Mike McDaniel,Alex Kemp,MIA00,Hard Rock Stadium
6188,2022_04_BUF_BAL,2022,REG,4,2022-10-02,Sunday,13:00,BUF,23.0,BAL,...,NaN,00-0034857,00-0034796,Josh Allen,Lamar Jackson,Sean McDermott,John Harbaugh,Jerome Boger,BAL00,M&T Bank Stadium
6228,2022_06_BUF_KC,2022,REG,6,2022-10-16,Sunday,16:25,BUF,24.0,KC,...,NaN,00-0034857,00-0033873,Josh Allen,Patrick Mahomes,Sean McDermott,Andy Reid,Brad Allen,KAN00,GEHA Field at Arrowhead Stadium


In [9]:
if normalizacion == 1:
    df['FTHG'] = (df['FTHG'] - df['FTHG'].min()) / ( df['FTHG'].max() - df['FTHG'].min()) * MAX
    df['FTAG'] = (df['FTAG'] - df['FTAG'].min()) / ( df['FTAG'].max() - df['FTAG'].min()) * MAX

In [10]:
df[["FTHG", "FTAG"]].mean()

FTHG    4.390432
FTAG    4.603175
dtype: float64

In [11]:
df[['FTHG','FTAG']]

,FTHG,FTAG
6137,6.458333,1.666667
6138,5.625000,5.476190
6139,5.416667,5.000000
6140,2.083333,3.809524
6141,4.791667,4.047619
...,...,...
6240,4.166667,8.333333
6241,7.708333,4.761905
6242,9.166667,4.761905
6243,2.083333,3.095238


In [12]:
pr = nfl.import_schedules([temporada])
pr = pr[pr['week']==semana ]
juegos = pr['week'].count()

In [13]:
pr = pr.rename(columns={'away_team':'AwayTeam',
                                   'home_team':'HomeTeam',
                       'away_score':'FTHG',
                       'home_score':'FTAG'})

In [14]:
model = pb.models.RueSalvesenGoalModel(df["FTHG"], df["FTAG"], df["HomeTeam"], df["AwayTeam"])
model.fit()

In [15]:
print('Rus Salvesen Model')
print(f'Season: {temporada}')
print(f'Week: {semana}')
for i in range(0,juegos):
    EL = pr.iloc[i]['HomeTeam']
    EV = pr.iloc[i]['AwayTeam']
    probs = model.predict(EL, EV)
    if probs.home_win >= .50 :
        print(f'L {EL} -{100/probs.home_win:.0f} vs V {EV} +{100/probs.away_win:.0f}')
    elif probs.away_win >= .50:
        print(f'L {EL} +{100/probs.home_win:.0f} vs V {EV} -{100/probs.away_win:.0f}')
    else:
        print(f'L {EL} +{100/probs.home_win:.0f} vs V {EV} +{100/probs.away_win:.0f}')

Rus Salvesen Model
Season: 2022
Week: 8
L TB -188 vs V BAL +330
L JAX +415 vs V DEN -174
L ATL +276 vs V CAR +202
L DAL +558 vs V CHI -148
L DET -153 vs V MIA +438
L MIN +321 vs V ARI -186
L NO +244 vs V LV +214
L NYJ +249 vs V NE +230
L PHI +833 vs V PIT -128
L HOU -181 vs V TEN +335
L IND +204 vs V WAS +286
L LA -171 vs V SF +390
L SEA -153 vs V NYG +446
L BUF +1010 vs V GB -124
L CLE -161 vs V CIN +393


In [16]:
model

Module: Penaltyblog

Model: Rue Salvesen

Number of parameters: 67
Log Likelihood: -386.777
AIC: 907.554

Team                 Attack               Defence             
------------------------------------------------------------
ARI                  1.01                 0.135               
ATL                  1.179                0.406               
BAL                  0.995                0.513               
BUF                  0.468                0.472               
CAR                  1.006                0.187               
CHI                  0.961                0.081               
CIN                  0.879                0.432               
CLE                  1.215                0.383               
DAL                  0.595                0.328               
DEN                  0.695                -0.196              
DET                  1.33                 0.283               
GB                   1.126                0.201               
HOU           

In [17]:
model.get_params()

{'attack_ARI': 1.010148473469816,
 'attack_ATL': 1.1789258270257423,
 'attack_BAL': 0.9946367434387047,
 'attack_BUF': 0.4679840271360171,
 'attack_CAR': 1.0062087009502956,
 'attack_CHI': 0.9612194047206196,
 'attack_CIN': 0.878956745013376,
 'attack_CLE': 1.2145753996009625,
 'attack_DAL': 0.5948033006489089,
 'attack_DEN': 0.6953923767691648,
 'attack_DET': 1.3300188891702112,
 'attack_GB': 1.1259608702012485,
 'attack_HOU': 1.1035893419691059,
 'attack_IND': 1.0626001348777752,
 'attack_JAX': 0.8765919713647848,
 'attack_KC': 1.3539222763368715,
 'attack_LA': 1.0530469599802927,
 'attack_LAC': 1.2378597068173054,
 'attack_LV': 1.376088793707313,
 'attack_MIA': 0.9659546179245373,
 'attack_MIN': 1.0211362757017148,
 'attack_NE': 0.8747300352712243,
 'attack_NO': 1.330742364115029,
 'attack_NYG': 0.9765450227708263,
 'attack_NYJ': 0.9440568610490921,
 'attack_PHI': 0.796214642581422,
 'attack_PIT': 1.0550322146521982,
 'attack_SEA': 1.1664942703091192,
 'attack_SF': 0.690353053015312